In [ ]:
import xarray as xr
import numpy as np
import torch
import pickle
import subprocess
import sys
import warnings
from pathlib import Path
from typing import Dict, Tuple, Optional

# 忽略NumPy DeprecationWarning
warnings.filterwarnings("ignore", category=DeprecationWarning, module="numpy.core")

# 确保安装必要的依赖库
def install_required_packages():
    required_packages = ['netCDF4', 'h5netcdf', 'scipy']
    for pkg in required_packages:
        try:
            __import__(pkg)
        except ImportError:
            print(f"安装缺失的依赖库: {pkg}")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", pkg],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.STDOUT
            )

# 加载表面变量数据（确保纬度严格递减且为模型兼容维度）
def load_surface_data(file_path: str) -> Tuple[Dict[str, torch.Tensor], xr.Dataset]:
    try:
        engines = ['netcdf4', 'h5netcdf', 'scipy']
        ds = None
        for engine in engines:
            try:
                ds = xr.open_dataset(file_path, engine=engine)
                break
            except:
                continue
        if ds is None:
            raise RuntimeError("无法加载表面变量数据")
        
        print(f"成功加载表面变量数据，包含变量: {list(ds.variables.keys())}")
        
        # 确保纬度严格递减
        if not np.all(np.diff(ds.latitude.values) < 0):
            print("表面数据纬度不是严格递减，正在翻转...")
            ds = ds.reindex(latitude=ds.latitude[::-1])
        

        if len(ds.latitude) == 721:
            ds = ds.isel(latitude=slice(0, 720))
        
        # 变量映射（原始变量名 -> 模型所需变量名）
        var_mapping = {
            'u10': '10u',
            'v10': '10v',
            't2m': '2t',
            'msl': 'msl'
        }
        
        # 提取并预处理变量
        surf_vars = {}
        for raw_var, model_var in var_mapping.items():
            if raw_var not in ds.variables:
                raise ValueError(f"表面数据缺少必要变量: {raw_var}")
            
            data = ds[raw_var].values
            if data.ndim < 3:
                raise ValueError(f"表面变量 {raw_var} 维度不正确（期望至少3维，实际{data.ndim}维）")
            
            # 处理时间维度（取最后两个时间步）
            if data.shape[0] >= 2:
                data = data[-2:, ...]  # 形状: (time, lat, lon)
            else:
                data = np.repeat(data, 2, axis=0)[:2, ...]
            
            # 转换为张量 (b, t, h, w)
            tensor = torch.from_numpy(data[None, ...].copy())
            surf_vars[model_var] = tensor
            print(f"表面变量 {model_var} 维度: {tensor.shape}")
        
        return surf_vars, ds
    except Exception as e:
        raise RuntimeError(f"加载表面变量数据失败: {str(e)}")

# 加载高空变量数据（处理纬度顺序和压力维度）
def load_atmospheric_data(file_path: str) -> Tuple[Dict[str, torch.Tensor], Tuple[int, ...]]:
    install_required_packages()
    
    engines = ['netcdf4', 'h5netcdf', 'scipy']
    air_ds = None
    
    for engine in engines:
        try:
            print(f"尝试使用{engine}引擎加载高空数据...")
            if engine == 'h5netcdf':
                air_ds = xr.open_dataset(file_path, engine=engine, invalid_netcdf=True)
            else:
                air_ds = xr.open_dataset(file_path, engine=engine)
            print(f"成功使用{engine}引擎加载高空数据")
            break
        except Exception as e:
            print(f"使用{engine}引擎加载失败: {str(e)}")
            continue
    
    if air_ds is None:
        raise RuntimeError("所有可用引擎都无法加载高空数据")
    
    print(f"高空变量数据包含变量: {list(air_ds.variables.keys())}")
    
    # 处理压力水平维度
    required_levels = (50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000)
    if 'pressure_level' in air_ds.dims:
        air_ds = air_ds.rename({'pressure_level': 'level'})
    elif 'level' not in air_ds.dims:
        level_dims = [dim for dim in air_ds.dims if 'level' in dim]
        if not level_dims:
            raise ValueError("高空数据缺少压力水平维度")
        air_ds = air_ds.rename({level_dims[0]: 'level'})
    
    # 对齐压力水平
    ds_levels = set(air_ds.level.values.astype(int))
    valid_levels = [l for l in required_levels if l in ds_levels]
    if not valid_levels:
        raise ValueError("没有找到有效的压力水平")
    air_ds = air_ds.sel(level=valid_levels)
    
    # 确保纬度严格递减且与表面数据维度一致
    if not np.all(np.diff(air_ds.latitude.values) < 0):
        print("高空数据纬度不是严格递减，正在翻转...")
        air_ds = air_ds.reindex(latitude=air_ds.latitude[::-1])
    

    if len(air_ds.latitude) == 721:
        air_ds = air_ds.isel(latitude=slice(0, 720))
    
    # 预处理高空变量
    atmos_vars = {}
    required_atmos_vars = ['t', 'u', 'v', 'q', 'z']
    for var in required_atmos_vars:
        if var not in air_ds.variables:
            raise ValueError(f"高空数据缺少必要变量: {var}")
        
        data = air_ds[var].values  # 维度: (time, level, lat, lon)
        if data.ndim < 4:
            raise ValueError(f"高空变量 {var} 维度不正确（期望至少4维，实际{data.ndim}维）")
        
        # 处理时间维度
        if data.shape[0] >= 2:
            data = data[-2:, ...]
        else:
            data = np.repeat(data, 2, axis=0)[:2, ...]
        
        # 转换为张量 (b, t, c, h, w)
        tensor = torch.from_numpy(data[None, ...].copy())
        atmos_vars[var] = tensor
        print(f"高空变量 {var} 维度: {tensor.shape}")
    
    return atmos_vars, tuple(valid_levels)

# 加载静态变量（确保与目标经纬度维度严格匹配）
def load_static_data(file_path: str, target_lat: np.ndarray, target_lon: np.ndarray) -> Dict[str, torch.Tensor]:
    try:
        if not Path(file_path).exists():
            raise FileNotFoundError(f"静态变量文件不存在: {file_path}")
        
        with open(file_path, 'rb') as f:
            static_data = pickle.load(f)
        
        required_static_vars = ['lsm', 'slt', 'z']
        missing_vars = [var for var in required_static_vars if var not in static_data]
        if missing_vars:
            raise ValueError(f"静态数据缺少必要变量: {missing_vars}")
        
        # 目标维度（已同步为720）
        target_lat_len = len(target_lat)
        target_lon_len = len(target_lon)
        target_shape = (target_lat_len, target_lon_len)
        print(f"目标静态变量维度: {target_shape}")
        
        # 处理静态变量
        static_vars = {}
        for var in required_static_vars:
            data = static_data[var]
            current_shape = data.shape[-2:]
            
            if current_shape != target_shape:
                from scipy.ndimage import zoom
                # 计算精确缩放因子
                zoom_factor = (
                    target_lat_len / current_shape[0],
                    target_lon_len / current_shape[1]
                )
                # 缩放并强制维度匹配目标
                data_zoomed = zoom(data, zoom_factor, order=1)  # 双线性插值
                data_zoomed = data_zoomed[:target_lat_len, :target_lon_len]  # 精确裁剪
                data = data_zoomed
            
            tensor = torch.from_numpy(data.copy())
            static_vars[var] = tensor
            print(f"静态变量 {var} 维度: {tensor.shape}")
        
        return static_vars
    except Exception as e:
        raise RuntimeError(f"加载静态变量数据失败: {str(e)}")

# 主函数
def main():
    # 配置文件路径,此处替换为自己的路径
    surface_file_path = ""
    atmospheric_file_path = ""
    static_file_path = ""
    model_checkpoint_path = ""
    output_dir = Path("")
    
    # 确保输出目录存在，如果不存在则创建
    if not output_dir.exists():
        output_dir.mkdir(parents=True, exist_ok=True)
        print(f"已创建输出目录: {output_dir}")
    else:
        print(f"输出目录已存在: {output_dir}")
    
    try:
        # 加载数据
        surf_vars, surf_ds = load_surface_data(surface_file_path)
        atmos_vars, atmos_levels = load_atmospheric_data(atmospheric_file_path)
        
        # 提取目标经纬度（已同步为720）
        target_lat = surf_ds.latitude.values
        target_lon = surf_ds.longitude.values
        lat_len = len(target_lat)
        lon_len = len(target_lon)
        print(f"目标经纬度长度: 纬度={lat_len}, 经度={lon_len}")
        
        # 加载静态变量（确保维度匹配）
        static_vars = load_static_data(
            file_path=static_file_path,
            target_lat=target_lat,
            target_lon=target_lon
        )
        
        # 校验所有变量的空间维度一致性
        for var, tensor in surf_vars.items():
            if tensor.shape[-2] != lat_len or tensor.shape[-1] != lon_len:
                raise ValueError(f"表面变量 {var} 空间维度不匹配（期望({lat_len},{lon_len})，实际{tensor.shape[-2:]}）")
        
        for var, tensor in atmos_vars.items():
            if tensor.shape[-2] != lat_len or tensor.shape[-1] != lon_len:
                raise ValueError(f"高空变量 {var} 空间维度不匹配（期望({lat_len},{lon_len})，实际{tensor.shape[-2:]}）")
        
        for var, tensor in static_vars.items():
            if tensor.shape[-2] != lat_len or tensor.shape[-1] != lon_len:
                raise ValueError(f"静态变量 {var} 空间维度不匹配（期望({lat_len},{lon_len})，实际{tensor.shape[-2:]}）")
        
        # 准备输入批次
        from aurora import Batch, Metadata
        batch = Batch(
            surf_vars=surf_vars,
            static_vars=static_vars,
            atmos_vars=atmos_vars,
            metadata=Metadata(
                lat=torch.from_numpy(target_lat.copy()),  # 已确保纬度递减且为720
                lon=torch.from_numpy(target_lon.copy()),
                #-1是有效时间的最后一个时间点，可以修改起始时间，-n表示倒数第n个，有多少个时间点取决于官网下载的数据选择了几天、几个小时。
                time=(np.datetime64(surf_ds.valid_time.values[-1]).astype('datetime64[s]').item(),),
                #一定注意不要越界：比如一共只有99个时间点，起始时间点就不能是-100。
                atmos_levels=atmos_levels
            )
        )
        
        # 加载模型（使用本地checkpoint）
        from aurora import AuroraPretrained  # 对应0.25°预训练模型
        model = AuroraPretrained()
        model.load_checkpoint_local(model_checkpoint_path)
        model.eval()  # 设置为评估模式
        
        # 模型预测
        with torch.inference_mode():
            prediction = model.forward(batch)
        
        # 保存预测结果
        surf_output_file = output_dir / "surface.nc" #文件名称可更改
        atmos_output_file = output_dir / "air.nc" #文件名称可更改
        
        # 处理表面变量预测
        def process_surf_pred(var):
            pred_tensor = prediction.surf_vars[var]
            # 形状应为 (b, t, h, w)，挤压为 (h, w)
            pred_np = pred_tensor.squeeze().numpy()
            # 确保空间维度匹配
            if pred_np.shape[0] != lat_len or pred_np.shape[1] != lon_len:
                # 若不匹配，使用插值调整
                from scipy.ndimage import zoom
                zoom_factor = (
                    lat_len / pred_np.shape[0],
                    lon_len / pred_np.shape[1]
                )
                pred_np = zoom(pred_np, zoom_factor, order=1)
                print(f"表面预测变量 {var} 维度不匹配，已插值为({lat_len},{lon_len})")
            # 添加时间维度 (1, h, w)
            return np.expand_dims(pred_np, axis=0)
        
        pred_2t = process_surf_pred("2t")
        pred_10u = process_surf_pred("10u")
        pred_10v = process_surf_pred("10v")
        pred_msl = process_surf_pred("msl")
        
        # 构建表面变量输出数据集
        surf_pred_ds = xr.Dataset(
            data_vars={
                "2t_pred": (["time", "latitude", "longitude"], pred_2t),
                "10u_pred": (["time", "latitude", "longitude"], pred_10u),
                "10v_pred": (["time", "latitude", "longitude"], pred_10v),
                "msl_pred": (["time", "latitude", "longitude"], pred_msl)
            },
            coords={
                #跟上面对应，surf_ds.valid_time.values[-1]：这里与上面选择的起始时间相同，是倒数第一个时间点
                #np.timedelta64(6, 'h')]，这里的6表示推测的是往后6h的气象数据，比如23h到5h，可以修改
                #但模型的有效预测能力受限于其设计的短期范围，超出后准确性会大幅下降，并非 “任意时间都能得到可靠结果”。
                "time": [np.datetime64(surf_ds.valid_time.values[-1]) + np.timedelta64(6, 'h')],
                "latitude": target_lat,
                "longitude": target_lon
            }
        )
        surf_pred_ds.to_netcdf(surf_output_file)
        print(f"表面变量预测结果已保存至: {surf_output_file}")
        
        # 处理高空变量预测
        def process_atmos_pred(var):
            pred_tensor = prediction.atmos_vars[var]
            # 形状应为 (b, t, c, h, w)，挤压为 (c, h, w)
            pred_np = pred_tensor.squeeze().numpy()
            # 确保空间维度匹配
            if pred_np.shape[-2] != lat_len or pred_np.shape[-1] != lon_len:
                from scipy.ndimage import zoom
                # 计算缩放因子（保留压力水平维度）
                zoom_factor = (1,  # 压力水平维度不缩放
                               lat_len / pred_np.shape[-2],
                               lon_len / pred_np.shape[-1])
                pred_np = zoom(pred_np, zoom_factor, order=1)
                print(f"高空预测变量 {var} 维度不匹配，已插值为({pred_np.shape})")
            # 添加时间维度 (1, c, h, w)
            return np.expand_dims(pred_np, axis=0)
        
        # 提取所有高空变量预测
        atmos_vars_list = ['t', 'u', 'v', 'q', 'z']
        atmos_preds = {var: process_atmos_pred(var) for var in atmos_vars_list}
        
        # 获取预测的压力水平并转换为numpy数组（关键修复）
        pred_levels = np.array(prediction.metadata.atmos_levels, dtype=int)
        
        # 构建高空变量输出数据集
        atmos_pred_ds = xr.Dataset(
            data_vars={
                f"{var}_pred": (["time", "level", "latitude", "longitude"], atmos_preds[var])
                for var in atmos_vars_list
            },
            coords={
                #跟上面对应，surf_ds.valid_time.values[-1]：这里与上面选择的起始时间相同，是倒数第一个时间点
                #np.timedelta64(6, 'h')]，这里的6表示推测的是往后6h的气象数据，比如23h到5h，可以修改
                #但模型的有效预测能力受限于其设计的短期范围，超出后准确性会大幅下降，并非 “任意时间都能得到可靠结果”。
                "time": [np.datetime64(surf_ds.valid_time.values[-1]) + np.timedelta64(6, 'h')],
                "level": pred_levels,  # 使用numpy数组而非元组
                "latitude": target_lat,
                "longitude": target_lon
            }
        )
        atmos_pred_ds.to_netcdf(atmos_output_file)
        print(f"高空变量预测结果已保存至: {atmos_output_file}")
        
    except Exception as e:
        print(f"执行出错: {str(e)}")

if __name__ == "__main__":
    main()
